#Split string into columns - pwc (PySpark)

In [0]:
dbutils.widgets.text('module', '4', 'define_module')

In [0]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark.sql.types import StructField, StructType, StringType, IntegerType
from pyspark.sql.window import Window as W
spark = SparkSession.builder.appName('practice').getOrCreate()

In [0]:
data=[('Biki_8090935525_Chandigarh_123_Haryana_Raveen_9876532513_Punjab_345_Punjab_Rishabh_9087638192_Faridabad_678_UP'),]
schema=StructType([StructField('users', StringType(), True)])
df=spark.createDataFrame(data,schema=schema)
df.show(truncate=False)

In [0]:
df=df.withColumn('users', F.split('users', '_'))
df.show(truncate=False)

In [0]:
explode_df=df.select(F.posexplode('users').alias('pos', 'users'))
explode_df.show(truncate=False)

In [0]:
mod=dbutils.widgets.get('module')
print(mod)

In [0]:
fields_df=explode_df.withColumn('name', F.when(F.col('pos')%mod==0, F.col('users'))) \
    .withColumn('mobile', F.when(F.col('pos')%mod==1, F.col('users'))) \
        .withColumn('address', F.when(F.col('pos')%mod==2, F.col('users'))) \
            .withColumn('zipcode', F.when(F.col('pos')%mod==3, F.col('users'))) \
                .withColumn('state', F.when(F.col('pos')%mod==4, F.col('users'))) \
                    .withColumn('group_id', (F.col('pos')/mod).cast('int'))
fields_df.show(truncate=False)

In [0]:
final_df=fields_df.groupBy('group_id')\
    .agg(F.max('name').alias('name'),\
        F.max('mobile').alias('mobile'),\
            F.max('address').alias('addresss'),\
                F.max('zipcode').alias('zipcode'),\
                    F.max('state').alias('state')).drop('group_id')
final_df.show(truncate=False)